<a href="https://colab.research.google.com/github/harvey2phase/summer-research/blob/master/cnn/deficit_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Shape: [training_size, 50 * HISTORY]
TRAIN_SET = "prob_of_death_at_80_e5_train_set.txt"
TRAIN_LABELS = "prob_of_death_at_80_e5_train_labels.txt"
EVAL_SET = "prob_of_death_at_80_e5_eval_set.txt"
EVAL_LABELS = "prob_of_death_at_80_e5_eval_labels.txt"

HISTORY = 5

STEPS = 3 * 10 ** 5
LEARNING_RATE = 10 ** -4
DROPOUT_RATE = 0.4

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
PATH = "/content/gdrive/My Drive/"

In [0]:
from __future__ import absolute_import, division, print_function

#import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
#import keras

###Silence Warnings
- 0: all messages are logged (default behavior)
- 1: INFO messages are not printed
- 2: INFO and WARNING messages are not printed
- 3: INFO, WARNING, and ERROR messages are not printed

In [0]:
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

###Model

In [0]:
def deficit_cnn_model(features, labels, mode):

    ''' Input Layer '''
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    input_layer = tf.reshape(features, [-1, 50, HISTORY, 1])

    ''' Convolutional Layer #1 '''
    # Computes 32 features using a 5x5 filter with ReLU activation.
    #   Padding is added to preserve width and height.
    # Tensor Shape: [batch_size, 50, HISTORY, 32]
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32,
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.relu
    )

    ''' Pooling Layer #1 '''
    # Max pooling layer with a 2x2 filter and stride of 2
    # Tensor Shape: [batch_size, 25, HISTORY / 2, 32]
    pool1 = tf.layers.max_pooling2d(
        inputs = conv1,
        pool_size = [2, 2],
        strides = 2
    )

    ''' Convolutional Layer #2 '''
    # Computes 64 features using a 5x5 filter.
    #   Padding is added to preserve width and height.
    # Tensor Shape: [batch_size, 25, HISTORY / 2, 64]
    conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 64,
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.relu
    )

    ''' Pooling Layer #2 '''
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Tensor Shape: [batch_size, 12, HISTORY / 4, 64]
    pool2 = tf.layers.max_pooling2d(
        inputs = conv2,
        pool_size = [2, 2],
        strides = 2
    )

    ''' Flatten tensor into a batch of vectors '''
    # Output Tensor Shape: [batch_size, 12 * 62]
    pool2_flat = tf.reshape(pool2, [-1, 768])

    ''' Dense Layer '''
    # Densely connected layer with 1024 neurons
    # Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(
        inputs = pool2_flat,
        units = 1024,
        activation = tf.nn.relu
    )
    
    ''' Add dropout operation '''
    # 0.6 probability that element will be kept
    # Tensor Shape: [batch_size, 1024]
    dropout = tf.layers.dropout(
        inputs = dense,
        rate = DROPOUT_RATE,
        training = mode == tf.estimator.ModeKeys.TRAIN
    )


    ''' Logits layer '''
    # Input Tensor Shape: [batch_size, 1024]
    # Tensor Shape: [batch_size, 1024]
    logits = tf.layers.dense(
        inputs = dropout,
        units = 10
    )

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input = logits, axis = 1),

        # Add `softmax_tensor` to the graph.
        # It is used for PREDICT and by the `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name = "softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode = mode,
            predictions = predictions
        )

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(
        labels = labels,
        logits = logits
    )

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:

        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate = LEARNING_RATE
        )
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step()
        )

        return tf.estimator.EstimatorSpec(
            mode = mode,
            loss = loss,
            train_op = train_op
        )

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels = labels,
            predictions = predictions["classes"]
        )
    }

    return tf.estimator.EstimatorSpec(
        mode = mode,
        loss = loss,
        eval_metric_ops = eval_metric_ops
    )

##Load Data and Setup Logging

###Data

In [0]:
train_data = np.loadtxt(PATH + TRAIN_SET)
eval_data = np.loadtxt(PATH + EVAL_SET)

train_labels = np.loadtxt(
    PATH + TRAIN_LABELS,
    dtype = np.int32
)
eval_labels = np.loadtxt(
    PATH + EVAL_LABELS,
    dtype = np.int32
)

###Logging

In [0]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors = tensors_to_log,
    every_n_iter = 50
)

##Create Estimator and Train

###Create Estimator

In [9]:
classifier = tf.estimator.Estimator(model_fn = deficit_cnn_model)

W0623 20:54:51.250265 140406964651904 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpg5qmu5wp


###Training

In [0]:
# Shape: [606, 50, 5, 1]
input_layer = tf.reshape(train_data, [-1, 50, HISTORY, 1])

In [11]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = train_data,
    y = train_labels,
    batch_size = 10,
    num_epochs = None,
    shuffle = True
)

classifier.train(
    input_fn = train_input_fn,
    steps = STEPS,
    hooks = [logging_hook]
)

W0623 20:54:51.510929 140406964651904 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0623 20:54:51.527864 140406964651904 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0623 20:54:51.530853 140406964651904 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_functions.py:500: add_

##Evaluation

In [12]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = eval_data,
    y = eval_labels,
    num_epochs = 1,
    shuffle = False
)
eval_results = classifier.evaluate(input_fn = eval_input_fn)
print(eval_results)


W0623 21:15:42.875232 140406964651904 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


{'accuracy': 0.6409, 'loss': 0.6384239, 'global_step': 300000}
